In [1]:
# Cell 1: Import libraries and set up directories
import os
import cv2
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Cell 2: Extract frames from videos
def extract_frames(video_path, output_folder, label):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_name = os.path.join(output_folder, f"{label}_{frame_count}.jpg")
        cv2.imwrite(frame_name, frame)
        frame_count += 1
    
    cap.release()
    cv2.destroyAllWindows()

video_folders = {
    "violence": "Dataset/Violence",
    "non_violence": "Dataset/NonViolence"
}

output_folder = "extract"

for label, folder in video_folders.items():
    for video_file in os.listdir(folder):
        video_path = os.path.join(folder, video_file)
        extract_frames(video_path, os.path.join(output_folder, label), label)


In [3]:
# Cell 3: Extract frames from videos
def prepare_yolo_data(frame_folder, labels):
    image_files = []
    for label in labels:
        label_folder = os.path.join(frame_folder, label)
        for file in os.listdir(label_folder):
            if file.endswith(".jpg"):
                image_files.append((os.path.join(label_folder, file), labels.index(label)))

    df = pd.DataFrame(image_files, columns=["image_path", "label"])
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"])

    def save_labels(df, split):
        os.makedirs(f"data/{split}/images", exist_ok=True)
        os.makedirs(f"data/{split}/labels", exist_ok=True)

        for _, row in df.iterrows():
            image_path, label = row["image_path"], row["label"]
            base_name = os.path.basename(image_path)
            new_image_path = f"data/{split}/images/{base_name}"
            label_path = new_image_path.replace("images", "labels").replace(".jpg", ".txt")

            # Check if the file already exists
            if os.path.exists(new_image_path):
                print(f"File {new_image_path} already exists. Skipping.")
                continue

            # Move image
            try:
                shutil.move(image_path, new_image_path)
            except FileExistsError:
                print(f"File {new_image_path} already exists. Skipping.")
                continue

            # Create label file
            with open(label_path, "w") as f:
                f.write(f"{label} 0.5 0.5 1 1\n")  # YOLO format: class x_center y_center width height

    save_labels(train_df, "train")
    save_labels(val_df, "val")

prepare_yolo_data("extract", ["non_violence", "violence"])

In [14]:
# Cell 4: Save data configuration
data_config = {
    'path': 'data',  # base path to data folder
    'train': 'train/images',  # train images
    'val': 'val/images',  # val images
    'names': ['non_violence', 'violence']  # class names
}

import yaml

with open('data_config.yaml', 'w') as f:
    yaml.dump(data_config, f)

In [1]:
# Cell 5: Train YOLOv8 model
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # you can use 'yolov8s.pt' for a larger model

# Train model
model.train(data='data_config.yaml', epochs=2, imgsz=640)

# Save model
model.save('yolov8_violence_detection.pt')

New https://pypi.org/project/ultralytics/8.2.26 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.17 🚀 Python-3.12.1 torch-2.2.0+cpu CPU (12th Gen Intel Core(TM) i7-1255U)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data_config.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fram

train: Scanning C:\Users\sonit\OneDrive\Desktop\Work\Coding\Violence-Detection\data\train\labels... 13335 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13335/13335 [00:17<00:00, 777.06it/s]


train: New cache created: C:\Users\sonit\OneDrive\Desktop\Work\Coding\Violence-Detection\data\train\labels.cache


val: Scanning C:\Users\sonit\OneDrive\Desktop\Work\Coding\Violence-Detection\data\val\labels... 3334 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3334/3334 [00:04<00:00, 740.38it/s]


val: New cache created: C:\Users\sonit\OneDrive\Desktop\Work\Coding\Violence-Detection\data\val\labels.cache
Plotting labels to runs\detect\train8\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train8
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        2/2         0G     0.1516     0.2157     0.8966         28        640: 100%|██████████| 834/834 [1:12:04<00:00,  5.18s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 105/105 [03:54<00:00,  2.23s/it]

                   all       3334       3334      0.999          1      0.995      0.995



2 epochs completed in 2.530 hours.
Optimizer stripped from runs\detect\train8\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train8\weights\best.pt, 6.2MB

Validating runs\detect\train8\weights\best.pt...
Ultralytics YOLOv8.1.17 🚀 Python-3.12.1 torch-2.2.0+cpu CPU (12th Gen Intel Core(TM) i7-1255U)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 105/105 [03:22<00:00,  1.93s/it]


                   all       3334       3334      0.999          1      0.995      0.995
          non_violence       3334       1079          1          1      0.995      0.995
              violence       3334       2255      0.998      0.999      0.995      0.995
Speed: 0.6ms preprocess, 49.4ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\train8
